In [1]:
import sys
import csv

from pathlib import Path

import psycopg2

import pandas as pd

sys.path.append('../../')
from utils import get_config

ERROR! Session/line number was not unique in database. History logging moved to new session 939


In [2]:
resource_p = Path("../resources")

# unhealthy food biz files
hdf_p = resource_p / "partial_unhealthy_food_biz.csv"
hdf_noise_p = resource_p / "noise_unhealthy_food_biz.csv"

# all food biz
all_food_p = resource_p / "partial_all_food_biz.csv"
noise_all_p = resource_p / "noise_all_food_biz.csv"

In [3]:
def read_categories_csv(p):
    with open(p) as csvfile:
        reader = csv.reader(csvfile)
        res = []
        for row in reader:
            res.append(row[0])
    return res

In [4]:
nourish_user = get_config("nourish_db","username")
nourish_pswd = get_config("nourish_db","passkey")

conn = psycopg2.connect(
    host="awesome-hw.sdsc.edu",
    database="nourish",
    user=nourish_user,
    password=nourish_pswd)

In [5]:
# Get resgistrant attributes
qry = """
WITH unnested_cats AS (SELECT unnest(categories) AS cats
FROM ca_business)
SELECT cats, count(cats) as category_count
FROM unnested_cats
GROUP BY cats
ORDER BY category_count DESC;
"""


cur = conn.cursor()

# execute a statement
cur.execute(qry)


# display the PostgreSQL database server version
res = cur.fetchall()
# res = [c[0] for c in col_names]
       
# Close the communication with the PostgreSQL
cur.close()

# create category df
categories_df = pd.DataFrame(res, columns=["categories", "category_count"])
categories_df

,categories,category_count
0,Restaurant,38523
1,Fast food restaurant,15560
2,Mexican restaurant,13969
3,Auto repair shop,13163
4,Beauty salon,10758
...,...,...
4015,Regional council,1
4016,Residential area,1
4017,Sheepskin and wool products supplier,1
4018,Ladder supplier,1


In [6]:
# read in hdf files
hdf_list = read_categories_csv(hdf_p)
hdf_noise_list = read_categories_csv(hdf_noise_p)

# https://stackoverflow.com/a/37864171
m = (categories_df["categories"].str.contains('|'.join(hdf_list), case=False)) | (categories_df["categories"] == "Bar")
filtered_hpf = categories_df.loc[m]

# Remove some noisy hits from matching on bar
filtered_hpf = filtered_hpf[~filtered_hpf["categories"].str.contains('|'.join(hdf_noise_list), case=False)]

filtered_hpf["categories"].to_csv(resource_p / "exact_unhealthy_categories.csv", index=False)

filtered_hpf.head()

,categories,category_count
1,Fast food restaurant,15560
16,Bar,8184
18,Pizza restaurant,7898
21,Hamburger restaurant,7281
22,Convenience store,7107


In [7]:
# make total food list
# read in hdf files
all_food_list = read_categories_csv(all_food_p)
noise_list = read_categories_csv(noise_all_p)

# combine hdf list and all food list
all_food_list = all_food_list + hdf_list
all_food_list = list(set(all_food_list))

m = (categories_df["categories"].str.contains('|'.join(all_food_list), case=False)) | (categories_df["categories"] == "Bar")
filtered_food = categories_df.loc[m]

# Remove some noisy hits from matching on bar
filtered_food = filtered_food[~filtered_food["categories"].str.contains('|'.join(noise_list), case=False)]

filtered_food["categories"].to_csv(resource_p / "exact_food_categories.csv", index=False)

filtered_food

/var/folders/wd/y5szpxf50s70ykknr5mg_1n00000gn/T/ipykernel_29725/2979570344.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m = (categories_df["categories"].str.contains('|'.join(all_food_list), case=False)) | (categories_df["categories"] == "Bar")


,categories,category_count
0,Restaurant,38523
1,Fast food restaurant,15560
2,Mexican restaurant,13969
5,Coffee shop,10487
7,Grocery store,10070
...,...,...
3952,Industrial supermarket,1
3962,Anago restaurant,1
3982,Chanko restaurant,1
3993,Alsace restaurant,1


In [8]:
# rewrite csv files
partial_df = pd.DataFrame(all_food_list)
partial_df.rename(columns={0: "categories"}, inplace=True)
partial_df.to_csv(resource_p / "partial_all_food_biz.csv", index=False, header=None)

noise_df = pd.DataFrame(noise_list)
noise_df.rename(columns={0: "categories"}, inplace=True)
noise_df.to_csv(resource_p / "noise_all_food_biz.csv", index=False, header=None)


partial_df = pd.DataFrame(hdf_list)
partial_df.rename(columns={0: "categories"}, inplace=True)
partial_df.to_csv(resource_p / "partial_unhealthy_food_biz.csv", index=False, header=None)

noise_df = pd.DataFrame(hdf_noise_list)
noise_df.rename(columns={0: "categories"}, inplace=True)
noise_df.to_csv(resource_p / "noise_unhealthy_food_biz.csv", index=False, header=None)

# Test lists via qry construction

## HDF QRY

In [9]:
# create where clause for query
qry_where_list = [f"'{c}' = any(categories) OR " for c in filtered_hpf["categories"].tolist()]
# clean up last or
qry_where_list[-1] = qry_where_list[-1].replace(" OR ", "")
qry_where = "WHERE (" + ''.join(qry_where_list) + ")"

qry = "WITH store_names AS (SELECT DISTINCT name AS dist_names " \
      "FROM ca_business " \
      f"{qry_where}) " \
      "SELECT COUNT(*) " \
      "as count " \
      "from ca_business inner join store_names on ca_business.name = store_names.dist_names;"
qry

"WITH store_names AS (SELECT DISTINCT name AS dist_names FROM ca_business WHERE ('Fast food restaurant' = any(categories) OR 'Bar' = any(categories) OR 'Pizza restaurant' = any(categories) OR 'Hamburger restaurant' = any(categories) OR 'Convenience store' = any(categories) OR 'Liquor store' = any(categories) OR 'Ice cream shop' = any(categories) OR 'Dessert shop' = any(categories) OR 'Pizza delivery' = any(categories) OR 'Bar & grill' = any(categories) OR 'Pizza Takeout' = any(categories) OR 'Chicken wings restaurant' = any(categories) OR 'Donut shop' = any(categories) OR 'Cocktail bar' = any(categories) OR 'Wine bar' = any(categories) OR 'Wine store' = any(categories) OR 'Sports bar' = any(categories) OR 'Brewery' = any(categories) OR 'Beer store' = any(categories) OR 'Candy store' = any(categories) OR 'Dessert restaurant' = any(categories) OR 'Chocolate shop' = any(categories) OR 'Hot dog restaurant' = any(categories) OR 'Brewpub' = any(categories) OR 'Pastry shop' = any(categories) 

In [10]:
cur = conn.cursor()

# execute a statement
cur.execute(qry)


# display the PostgreSQL database server version
res = cur.fetchall()
# res = [c[0] for c in col_names]
       
# Close the communication with the PostgreSQL
cur.close()
res[0]

(70914,)

In [11]:
qry = "WITH store_names AS " \
            "(SELECT DISTINCT name AS dist_names " \
             "FROM ca_business " \
        f"{qry_where}), " \
            "hpf_stores AS " \
                "(SELECT * FROM ca_business INNER JOIN store_names ON ca_business.name = store_names.dist_names) " \
        "SELECT COUNT(*) as count, array_agg(distinct name) as restaurants " \
        "FROM hpf_stores " \
        "WHERE name NOT IN (SELECT DISTINCT name FROM ca_business WHERE 'Supermarket' = any(categories));"
qry

"WITH store_names AS (SELECT DISTINCT name AS dist_names FROM ca_business WHERE ('Fast food restaurant' = any(categories) OR 'Bar' = any(categories) OR 'Pizza restaurant' = any(categories) OR 'Hamburger restaurant' = any(categories) OR 'Convenience store' = any(categories) OR 'Liquor store' = any(categories) OR 'Ice cream shop' = any(categories) OR 'Dessert shop' = any(categories) OR 'Pizza delivery' = any(categories) OR 'Bar & grill' = any(categories) OR 'Pizza Takeout' = any(categories) OR 'Chicken wings restaurant' = any(categories) OR 'Donut shop' = any(categories) OR 'Cocktail bar' = any(categories) OR 'Wine bar' = any(categories) OR 'Wine store' = any(categories) OR 'Sports bar' = any(categories) OR 'Brewery' = any(categories) OR 'Beer store' = any(categories) OR 'Candy store' = any(categories) OR 'Dessert restaurant' = any(categories) OR 'Chocolate shop' = any(categories) OR 'Hot dog restaurant' = any(categories) OR 'Brewpub' = any(categories) OR 'Pastry shop' = any(categories) 

In [12]:
cur = conn.cursor()

# execute a statement
cur.execute(qry)


# display the PostgreSQL database server version
res = cur.fetchall()
# res = [c[0] for c in col_names]
       
# Close the communication with the PostgreSQL
cur.close()
res[0][0]

68539

In [13]:
qry = "WITH store_names AS " \
            "(SELECT DISTINCT name AS dist_names " \
             "FROM ca_business " \
        f"{qry_where}), " \
            "hpf_stores AS " \
                "(SELECT * FROM ca_business INNER JOIN store_names ON ca_business.name = store_names.dist_names) " \
        "SELECT zip, count(*) as count, array_agg(distinct name) as restaurants " \
        "FROM hpf_stores " \
        "WHERE name NOT IN (SELECT DISTINCT name FROM ca_business WHERE 'Supermarket' = any(categories)) " \
        "group by zip " \
        "order by count desc, zip;"
qry

"WITH store_names AS (SELECT DISTINCT name AS dist_names FROM ca_business WHERE ('Fast food restaurant' = any(categories) OR 'Bar' = any(categories) OR 'Pizza restaurant' = any(categories) OR 'Hamburger restaurant' = any(categories) OR 'Convenience store' = any(categories) OR 'Liquor store' = any(categories) OR 'Ice cream shop' = any(categories) OR 'Dessert shop' = any(categories) OR 'Pizza delivery' = any(categories) OR 'Bar & grill' = any(categories) OR 'Pizza Takeout' = any(categories) OR 'Chicken wings restaurant' = any(categories) OR 'Donut shop' = any(categories) OR 'Cocktail bar' = any(categories) OR 'Wine bar' = any(categories) OR 'Wine store' = any(categories) OR 'Sports bar' = any(categories) OR 'Brewery' = any(categories) OR 'Beer store' = any(categories) OR 'Candy store' = any(categories) OR 'Dessert restaurant' = any(categories) OR 'Chocolate shop' = any(categories) OR 'Hot dog restaurant' = any(categories) OR 'Brewpub' = any(categories) OR 'Pastry shop' = any(categories) 

In [14]:
cur = conn.cursor()

# execute a statement
cur.execute(qry)


# display the PostgreSQL database server version
res = cur.fetchall()
# res = [c[0] for c in col_names]
       
# Close the communication with the PostgreSQL
cur.close()
pd.DataFrame(res, columns=["Zip", "Count", "Names"])

,Zip,Count,Names
0,92101.0,490,"[10 Barrel Brewing San Diego, 1919, 1st and Iv..."
1,94103.0,267,"[3rd Street Tap Room, 54 Mint Ristorante Itali..."
2,90028.0,263,"[101 Coffee Shop, 25 Degrees, 33 Taps Hollywoo..."
3,94110.0,253,"[20 Spot, 24th Street Bar, 49'ERS Liquors & Gr..."
4,94102.0,229,"[2G Japanese Brasserie, 620 Jones, 7-Eleven, A..."
...,...,...,...
1608,96124.0,1,[Lakeside Bar]
1609,96128.0,1,[Shell]
1610,96134.0,1,[Stateline General Store and Liquor]
1611,97701.0,1,[St Helena Wine Co]


## ALL BIZ QRY

In [15]:
# create where clause for query
# qry_where_list = [f"'{c}' = any(categories) OR " for c in filtered_food["categories"].tolist()]
qry_where_list = []
for c in filtered_food["categories"].tolist():
    
    if "'" in c:
        where_part = c.replace("'", "''")
        where_part = "'" + where_part + "' = any(categories) OR "
        # qry_where = qry_where + f"'{where_part}'"
    else:
        where_part = "'" + c + "' = any(categories) OR "
    qry_where_list.append(where_part)
# clean up last or
qry_where_list[-1] = qry_where_list[-1].replace(" OR ", "")

qry_where = "WHERE ("
for w in qry_where_list:
    if "Farmer" in w:
        print(fr"{w}")
    qry_where = fr"{qry_where}{w} "
qry_where = qry_where  + "))"
    
# qry_where = "WHERE (" + ''.join(qry_where_list) + ")"

# qry = "WITH store_names AS (SELECT DISTINCT name AS dist_names " \
#       "FROM ca_business " \
#       f"{qry_where}) " \
#       "SELECT COUNT(*) " \
#       "as count " \
#       "from ca_business inner join store_names on ca_business.name = store_names.dist_names;"

qry = """WITH store_names AS (SELECT DISTINCT name AS dist_names FROM ca_business %s SELECT COUNT(*) as count from ca_business inner join store_names on ca_business.name = store_names.dist_names;""" % (qry_where)
len(qry)

'Farmers'' market' = any(categories) OR 


20543

In [16]:
cur = conn.cursor()

# execute a statement
cur.execute(qry)


# display the PostgreSQL database server version
res = cur.fetchall()
# res = [c[0] for c in col_names]
       
# Close the communication with the PostgreSQL
cur.close()
res[0]

(153920,)

In [17]:
qry = "WITH store_names AS " \
            "(SELECT DISTINCT name AS dist_names " \
             "FROM ca_business " \
        f"{qry_where}, " \
            "hpf_stores AS " \
                "(SELECT * FROM ca_business INNER JOIN store_names ON ca_business.name = store_names.dist_names) " \
        "SELECT zip, count(*) as count, array_agg(distinct name) as restaurants " \
        "FROM hpf_stores " \
        "group by zip " \
        "order by count desc, zip;"
len(qry)

20669

In [18]:
cur = conn.cursor()

# execute a statement
cur.execute(qry)


# display the PostgreSQL database server version
res = cur.fetchall()
# res = [c[0] for c in col_names]
       
# Close the communication with the PostgreSQL
cur.close()
pd.DataFrame(res, columns=["Zip", "Count", "Names"])

,Zip,Count,Names
0,92101.0,932,"[1010 Caffe, 10 Barrel Brewing San Diego, 12th..."
1,94110.0,717,"[20 Spot, 23rd liquor store, 23rd & Mission Pr..."
2,94103.0,608,"[1601 Bar & Kitchen, 18 Rabbits, 3rd Street Ta..."
3,NaN,519,"[., 1105 Cooperative, 1675 E. Mariposa Rd. Sto..."
4,94109.0,499,"[1608 Bistro, 1760, 707 Sutter, 721 Lounge, 80..."
...,...,...,...
1811,98887.0,1,[Grumpy Bears Retreat]
1812,98940.0,1,[A Mi Estilo Mexican Restaurant]
1813,98960.0,1,[Reyes Market]
1814,99321.0,1,[Queen Bean Caffé Crafton Hills College]


In [12]:
# TODO since certain categories like liquor store also apply to grocery chains
# also create list of store names that should not be in HPF list
# ex: Safeway
other_list = ["grocery"]

m = (categories_df["categories"].str.contains('|'.join(other_list), case=False))
other_df = categories_df.loc[m]

# Remove some noisy hits from matching on bar
# filtered_food = filtered_food[~filtered_food["categories"].str.contains('|'.join(noise_list), case=False)]

# filtered_food["categories"].to_csv(resource_p / "exact_food_categories.csv", index=False)

other_df

,categories,category_count
7,Grocery store,10070
265,Grocery delivery service,1019
484,Mexican grocery store,479
514,Gourmet grocery store,447
543,Asian grocery store,421
818,Indian grocery store,241
1508,Korean grocery store,76
1534,Italian grocery store,73
1634,Japanese grocery store,61
2194,Kosher grocery store,27
